In [1]:
import ipytest
import pytest
import random
import numpy as np

ipytest.autoconfig()

## 基本ソートアルゴリズム

In [5]:
%%ipytest

# バブルソート
def bubble_sort(arr):
    for i in range(len(arr)):
        for j in range(i, len(arr)):
            if arr[i] > arr[j]:
                arr[i], arr[j] = arr[j], arr[i]

    return arr


@pytest.mark.parametrize(
    "arr, expected",
    [
        ([3, 2, 1], [1, 2, 3]),
        ([1, 2, 3], [1, 2, 3]),
        ([1], [1]),
        ([], []),
    ],
)

def test_bubble_sort(arr, expected):
    assert bubble_sort(arr) == expected

....                                                                                         [100%]
4 passed in 0.02s


In [8]:
%%ipytest

# コムソート
# バブルソートは隣り合う要素を比較するが、コムソートは間隔をあけて比較する
# そのため，後ろにある小さな要素が前に移動するのが早い
def comb_sort(arr):
    gap = len(arr) # 比較する要素の間隔
    shrink = 1.3 # 縮小率
    swapped = True # 交換が発生したかどうか: 交換が発生しなくなるor間隔が1になるとソート完了

    while gap > 1 or swapped:
        gap = int(gap / shrink) # int()で切り捨て
        if gap < 1:
            gap = 1

        swapped = False
        for i in range(len(arr) - gap):
            if arr[i] > arr[i + gap]:
                arr[i], arr[i + gap] = arr[i + gap], arr[i]
                swapped = True

    return arr

@pytest.mark.parametrize(
    "arr, expected",
    [
        ([3, 2, 1], [1, 2, 3]),
        ([1, 2, 3], [1, 2, 3]),
        ([1], [1]),
        ([], []),
    ],
)

def test_comb_sort(arr, expected):
    assert comb_sort(arr) == expected

....                                                                                         [100%]
4 passed in 0.03s


In [9]:
%%ipytest

# 選択ソート
def selection_sort(arr):
    for i in range(len(arr)):
        min_index = i
        for j in range(i + 1, len(arr)):
            if arr[j] < arr[min_index]:
                min_index = j
        arr[i], arr[min_index] = arr[min_index], arr[i]

    return arr

@pytest.mark.parametrize(
    "arr, expected",
    [
        ([3, 2, 1], [1, 2, 3]),
        ([1, 2, 3], [1, 2, 3]),
        ([1], [1]),
        ([], []),
    ],
)

def test_selection_sort(arr, expected):
    assert selection_sort(arr) == expected

....                                                                                         [100%]
4 passed in 0.04s


In [10]:
%%ipytest

# 挿入ソート
def insertion_sort(arr):
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j] > key:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key

    return arr

@pytest.mark.parametrize(
    "arr, expected",
    [
        ([3, 2, 1], [1, 2, 3]),
        ([1, 2, 3], [1, 2, 3]),
        ([1], [1]),
        ([], []),
    ],
)

def test_insertion_sort(arr, expected):
    assert insertion_sort(arr) == expected

....                                                                                         [100%]
4 passed in 0.05s


In [15]:
%%ipytest

def bubble_sort_kry(arr, key=lambda x: x):
    arr_key = [key(x) for x in arr]
    for i in range(len(arr)):
        for j in range(i, len(arr)):
            if arr_key[i] > arr_key[j]:
                arr[i], arr[j] = arr[j], arr[i]
                arr_key[i], arr_key[j] = arr_key[j], arr_key[i]

    return arr

@pytest.mark.parametrize(
    "arr, key, expected",
    [
        ([3, 2, 1], lambda x: x, [1, 2, 3]),
        ([1, 2, 3], lambda x: x, [1, 2, 3]),
        ([1], lambda x: x, [1]),
        ([], lambda x: x, []),
        ([3, 2, 1], lambda x: -x, [3, 2, 1]),
        ([1, 2, 3], lambda x: -x, [3, 2, 1]),
        ([1], lambda x: -x, [1]),
        ([], lambda x: -x, []),
    ],
)

def test_bubble_sort_kry(arr, key, expected):
    assert bubble_sort_kry(arr, key) == expected

........                                                                                     [100%]
8 passed in 0.04s


## 発展ソートアルゴリズム

In [22]:
%%ipytest

# Tim Sort
# https://qiita.com/nishiwakki/items/806c24b8d061083d0808
# マージソートと挿入ソートを組み合わせたアルゴリズム
# Pythonのsorted()関数やlist.sort()関数で使われている
def tim_sort(arr):
    if len(arr) < 2:
        return arr

    def _calc_minrun(n):
        r = 0
        while n >= 64:
            r |= n & 1 # nの最下位ビットをrの最下位ビットにコピー
            n >>= 1 # nを1ビット右シフト: nを2で割る
        return n + r

    def _insertion_sort(arr, left, right):
        for i in range(left + 1, right + 1):
            key = arr[i]
            j = i - 1
            while j >= left and arr[j] > key:
                arr[j + 1] = arr[j]
                j -= 1
            arr[j + 1] = key

    def _merge(arr, l, m, r):
        len1, len2 = m - l + 1, r - m
        left, right = [], []
        for i in range(len1):
            left.append(arr[l + i])
        for i in range(len2):
            right.append(arr[m + 1 + i])

        i, j, k = 0, 0, l
        while i < len1 and j < len2:
            if left[i] <= right[j]:
                arr[k] = left[i]
                i += 1
            else:
                arr[k] = right[j]
                j += 1
            k += 1

        while i < len1:
            arr[k] = left[i]
            i += 1
            k += 1

        while j < len2:
            arr[k] = right[j]
            j += 1
            k += 1
    minrun = _calc_minrun(len(arr)) # 最小の部分配列の要素数
    
    # 1:サブ配列に分割
    for i in range(0, len(arr), minrun):
        _insertion_sort(arr, i, min((i + minrun - 1), len(arr) - 1)) # 2: サブ配列内で挿入ソート
    
    # 3: サブ配列をマージ
    size = minrun
    while size < len(arr):
        for left in range(0, len(arr), 2 * size):
            mid = left + size - 1
            right = min((left + 2 * size - 1), len(arr) - 1)
            _merge(arr, left, mid, right)
        size *= 2

    return arr

a = list(range(1000))
random.shuffle(a)
@pytest.mark.parametrize(
    "arr, expected",
    [
        (a, list(range(1000))),
    ],
)

def test_tim_sort(arr, expected):
    assert tim_sort(arr) == expected

.                                                                                            [100%]
1 passed in 0.02s


In [23]:
%%ipytest

# シェルソート
def shell_sort(arr):
    n = len(arr)
    gap = n // 2

    while gap > 0:
        for i in range(gap, n):
            temp = arr[i]
            j = i
            while j >= gap and arr[j - gap] > temp:
                arr[j] = arr[j - gap]
                j -= gap
            arr[j] = temp
        gap //= 2

    return arr

@pytest.mark.parametrize(
    "arr, expected",
    [
        ([3, 2, 1], [1, 2, 3]),
        ([1, 2, 3], [1, 2, 3]),
        ([1], [1]),
        ([], []),
    ],
)

def test_shell_sort(arr, expected):
    assert shell_sort(arr) == expected

....                                                                                         [100%]
4 passed in 0.04s


In [5]:
%%ipytest

# ヒープソート
def heap_sort(arr):
    def _align_element(arr, start, end):
        """ヒープを構築

        Args:
            arr (_type_): 整列対象の配列
            start (_type_): 整列対象の開始インデックス
            end (_type_): 整列対象の終了インデックス
        """
        k = start
        
        while True:
            left = arr[2*k + 1] if 2*k + 1 < end else -np.inf
            right = arr[2*k + 2] if 2*k + 2 < end else -np.inf

            if left == -np.inf and right == -np.inf:
                break
            elif arr[k] >= max(left, right):
                break
            elif left > right:
                arr[k], arr[2*k + 1] = arr[2*k + 1], arr[k]
                k = 2*k + 1
            else:
                arr[k], arr[2*k + 2] = arr[2*k + 2], arr[k]
                k = 2*k + 2

    def _build_heap(arr):
        """ヒープを構築

        Args:
            arr (_type_): ヒープを構築する配列
        """
        for i in range(len(arr)//2, -1, -1):
            _align_element(arr, i, len(arr))

    def _delete_root(arr, end):
        """根を削除

        Args:
            arr (_type_): 根を削除する配列
            end (_type_): 現在の配列の終了インデックス
        """
        arr[0], arr[end] = arr[end], arr[0]
        _align_element(arr, 0, end)

    _build_heap(arr)
    for i in range(len(arr)-1, 0, -1):
        _delete_root(arr, i)

    return arr

@pytest.mark.parametrize(
    "arr, expected",
    [
        ([3, 2, 1], [1, 2, 3]),
        ([1, 2, 3], [1, 2, 3]),
        ([1], [1]),
        ([], []),
    ],
)

def test_heap_sort(arr, expected):
    assert heap_sort(arr) == expected


....                                                                                         [100%]
4 passed in 0.04s


In [7]:
%%ipytest

# マージソート
def merge_sort(arr):
    if len(arr) <= 1:
        return arr

    def _merge(left, right):
        result = []
        i = j = 0
        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
                j += 1

        result += left[i:]
        result += right[j:]

        return result

    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])

    return _merge(left, right)

@pytest.mark.parametrize(
    "arr, expected",
    [
        ([10, 9, 8, 7, 6, 5, 4, 3, 2, 1], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]),
        ([1, 2, 3], [1, 2, 3]),
        ([1], [1]),
        ([], []),
    ],
)

def test_merge_sort(arr, expected):
    assert merge_sort(arr) == expected


....                                                                                         [100%]
4 passed in 0.02s


In [2]:
%%ipytest

# クイックソート
def quick_sort(arr):
    if len(arr) <= 1:
        return arr

    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]

    return quick_sort(left) + middle + quick_sort(right)

@pytest.mark.parametrize(
    "arr, expected",
    [
        ([10, 9, 8, 7, 6, 5, 4, 3, 2, 1], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]),
        ([1, 2, 3], [1, 2, 3]),
        ([1], [1]),
        ([], []),
    ],
)

def test_quick_sort(arr, expected):
    assert quick_sort(arr) == expected

....                                                                                         [100%]
4 passed in 0.03s


## 探索アルゴリズム

In [3]:
%%ipytest

# 線形探索
def linear_search(arr, target):
    for i, x in enumerate(arr):
        if x == target:
            return i

    return -1

@pytest.mark.parametrize(
    "arr, target, expected",
    [
        ([1, 2, 3], 2, 1),
        ([1, 2, 3], 4, -1),
        ([1], 1, 0),
        ([], 1, -1),
    ],
)

def test_linear_search(arr, target, expected):
    assert linear_search(arr, target) == expected

....                                                                                         [100%]
4 passed in 0.03s


In [4]:
%%ipytest

# 二分探索
def binary_search(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1

    return -1

@pytest.mark.parametrize(
    "arr, target, expected",
    [
        ([1, 2, 3], 2, 1),
        ([1, 2, 3], 4, -1),
        ([1], 1, 0),
        ([], 1, -1),
    ],
)

def test_binary_search(arr, target, expected):
    assert binary_search(arr, target) == expected

....                                                                                         [100%]
4 passed in 0.03s


In [7]:
arr = [0, 2, 4, 1, 3]
sort_idx = sorted(range(len(arr)), key=lambda x: arr[x])
sort_arr = sorted(arr)

idx = binary_search(sort_arr, 2)
print(idx)
print(sort_idx[idx])

2
1


In [9]:
%%ipytest

# 二分探索木
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

class BinarySearchTree:
    def __init__(self):
        self.root = None

    def insert(self, value):
        if self.root is None:
            self.root = Node(value)
        else:
            self._insert(self.root, value)

    def _insert(self, node, value):
        if value < node.value:
            if node.left is None:
                node.left = Node(value)
            else:
                self._insert(node.left, value)
        else:
            if node.right is None:
                node.right = Node(value)
            else:
                self._insert(node.right, value)

    def search(self, value):
        return self._search(self.root, value)

    def _search(self, node, value):
        if node is None:
            return False
        elif node.value == value:
            return True
        elif value < node.value:
            return self._search(node.left, value)
        else:
            return self._search(node.right, value)

    def delete(self, value):
        self.root = self._delete(self.root, value)

    def _delete(self, node, value):
        if node is None:
            return None
        if value < node.value:
            node.left = self._delete(node.left, value)
        elif value > node.value:
            node.right = self._delete(node.right, value)
        else:
            if node.left is None:
                return node.right
            elif node.right is None:
                return node.left
            else:
                node.value = self._find_min(node.right).value
                node.right = self._delete(node.right, node.value)

        return node

    def _find_min(self, node):
        while node.left is not None:
            node = node.left
        return node

    def inorder(self):
        return self._inorder(self.root)

    def _inorder(self, node):
        if node is not None:
            self._inorder(node.left)
            print(node.value)
            self._inorder(node.right)

bst = BinarySearchTree()
bst.insert(3)
bst.insert(1)
bst.insert(4)
bst.insert(2)
bst.insert(5)
bst.inorder()

bst.delete(3)
bst.inorder()

@pytest.mark.parametrize(
    "arr, target, expected",
    [
        ([1, 2, 3], 2, True),
        ([1, 2, 3], 4, False),
        ([1], 1, True),
        ([], 1, False),
    ],
)

def test_binary_search_tree(arr, target, expected):
    bst = BinarySearchTree()
    for x in arr:
        bst.insert(x)
    assert bst.search(target) == expected

1
2
3
4
5
1
2
4
5
....                                                                                         [100%]
4 passed in 0.02s
